In [1]:
from modules.utils.schemas import AgentState, AthleteConditions, TrainingParams, PhysicalTest
from modules.utils.prompts import TEST_PROMPT_TEMPLATE


/home/jimmy.gomes/Documents/Projetos/Vida Pessoal/Treinador/src/modules/utils/prompts.py:29: SyntaxWarning: invalid escape sequence '\%'
  TEST_PROMPT_TEMPLATE = """


In [2]:
TrainingParams.model_fields

{'main_sport': FieldInfo(annotation=str, required=True),
 'training_modalities': FieldInfo(annotation=List[Literal['corrida', 'ciclismo', 'musculação', 'alongamento']], required=True),
 'objective': FieldInfo(annotation=str, required=True),
 'constraints': FieldInfo(annotation=List[str], required=True),
 'athlete_availability': FieldInfo(annotation=str, required=True),
 'suplement_already_taken': FieldInfo(annotation=List[str], required=True)}

In [3]:
athlete_conditions = AthleteConditions(
    height=180,
    weight=70,
    age=25,
    gender="masculino",
    body_type='ectomorfo',
    body_fat=.15)

training_params = TrainingParams(
    main_sport="motovelocidade",
    training_modalities = ['corrida', 'ciclismo', 'musculação'],
    objective = 'Aumentar a resistência para aguentar mais voltas na pista',
    constraints = ['Academia com pesos normais, sem equipamentos especiais', 'Possuo pouco alongamento'],
    athlete_availability = 'Segunda a Sexta, manhã e noite. Sábado apenas de manhã',
    suplement_already_taken = ['Creatina', 'Whey Protein', 'Multivitamínico', 'Termogênico']
)

prompt = TEST_PROMPT_TEMPLATE.format(main_sport=training_params.main_sport, objective=training_params.objective, main_sport_details=[], accepted_training_modalities=training_params.training_modalities)
print(prompt)

 
Você é um especialista em treinamento e fisiologia, focado no esporte motovelocidade.

Sua tarefa é fornecer um teste físico padronizado **com PELO MENOS 8 a 15 exercícios** para o macrociclo de treino, a fim de mensurar o avanço do atleta frente ao objetivo do treinamento, considerando o objetivo fornecido e as características do esporte.

Seu objetivo é permitir que o atleta seja capaz de avaliar seu progresso e ajustar seu treino conforme necessário.

Restrições:

1. Deve haver pelo menos 1 teste físico para cada grupamento muscular, conforme o objetivo do treinamento.
2. Deve haver entre 8 a 15 testes físicos no total
3. Somente pode have teste físico para as modalidades de treino aceitas.
4. Justifique a escolha o exercício como teste físico, e como o exercício poderá mensurar a evolução do atleta frente ao objetivo do macrociclo de treino.

O formato da resposta deve ser o seguinte:

{
    "test_name": "nome do teste físico",
    "test_exercises": "lista de exercícios do teste 

In [4]:
training_params

TrainingParams(main_sport='motovelocidade', training_modalities=['corrida', 'ciclismo', 'musculação'], objective='Aumentar a resistência para aguentar mais voltas na pista', constraints=['Academia com pesos normais, sem equipamentos especiais', 'Possuo pouco alongamento'], athlete_availability='Segunda a Sexta, manhã e noite. Sábado apenas de manhã', suplement_already_taken=['Creatina', 'Whey Protein', 'Multivitamínico', 'Termogênico'])

In [5]:
from dotenv import load_dotenv
from modules.utils.llm import ChatLLM
load_dotenv()

structured_llm = ChatLLM.with_structured_output(PhysicalTest)

In [6]:
x = structured_llm.invoke(prompt)
print(x)

test_name='Teste de Resistência e Força para Motovelocidade' test_exercises=[AerobicExercice(type='aerobic', modality='corrida longa', duration='30 minutos', description='Corrida contínua em ritmo moderado, visando manter a frequência cardíaca entre 70-80% da frequência máxima.', justification='A corrida longa é fundamental para aumentar a resistência aeróbica do atleta, permitindo que ele suporte mais tempo em alta intensidade durante as voltas na pista.'), AerobicExercice(type='aerobic', modality='ciclismo', duration='40 minutos', description='Ciclismo em terreno plano, mantendo uma cadência constante e moderada.', justification='O ciclismo é uma excelente forma de desenvolver resistência muscular e cardiovascular, essencial para a motovelocidade, onde a resistência é crucial para manter a performance ao longo da corrida.'), WeighedExercice(type='weighed', exercise='agachamento', series=1, repetitions='max', load='70% do RM', rest=60, justification='O agachamento é um exercício funda

In [7]:
print(x.test_name)

Teste de Resistência e Força para Motovelocidade


In [8]:
for i, exercise in enumerate(x.test_exercises):
    print(f"\n############### Exercício {i+1}: ######### ")
    print('\n')
    if exercise.type == "aerobic":
        print('Modalidade: ', exercise.modality)
        print('Duração: ', exercise.duration)
        print('Descrição: ', exercise.description)
        print('Justificativa: ', exercise.justification)
    elif exercise.type == "weighed":
        print('Exercício: ', exercise.exercise)
        print('Séries: ', exercise.series)
        print('Repetições: ', exercise.repetitions)
        print('Carga: ', exercise.load)
        print('Descanso: ', exercise.rest)
        print('Justificativa: ', exercise.justification)
    elif exercise.type == "stretching":
        print('Exercício: ', exercise.exercise)
        print('Duração: ', exercise.duration)
        print('Descrição: ', exercise.description)
        print('Justificativa: ', exercise.justification)


############### Exercício 1: ######### 


Modalidade:  corrida longa
Duração:  30 minutos
Descrição:  Corrida contínua em ritmo moderado, visando manter a frequência cardíaca entre 70-80% da frequência máxima.
Justificativa:  A corrida longa é fundamental para aumentar a resistência aeróbica do atleta, permitindo que ele suporte mais tempo em alta intensidade durante as voltas na pista.

############### Exercício 2: ######### 


Modalidade:  ciclismo
Duração:  40 minutos
Descrição:  Ciclismo em terreno plano, mantendo uma cadência constante e moderada.
Justificativa:  O ciclismo é uma excelente forma de desenvolver resistência muscular e cardiovascular, essencial para a motovelocidade, onde a resistência é crucial para manter a performance ao longo da corrida.

############### Exercício 3: ######### 


Exercício:  agachamento
Séries:  1
Repetições:  max
Carga:  70% do RM
Descanso:  60
Justificativa:  O agachamento é um exercício fundamental para fortalecer os músculos das pernas, que 